In [2]:
import os
import random
import cv2 as cv
import numpy as np

def bgr2rgb(img):
    return img[...,::-1]

def load_color_img(path):
    return np.array( bgr2rgb(cv.imread(path)) )

def filter_image(img):
    clahe = cv.createCLAHE(clipLimit=5.0, tileGridSize=(30,30))
    new = clahe.apply(img)
    clahe = cv.createCLAHE(clipLimit=2.0, tileGridSize=(10, 10))
    new = clahe.apply(new)
    new = cv.fastNlMeansDenoising(new, None, 30, 7, 21)
    return new

def load_gray_img(path):
    return cv.imread(path, 0)

#random.seed(3)

data_path = os.getcwd() + "/eye_data" + "/set1"
eyes_path = data_path + "/images" + "/"
vessels_path = data_path + "/vessels" + "/"

yes_path = data_path + "/classes/yes_filter" + "/"
no_path = data_path + "/classes/no_filter" + "/"

yeses = os.listdir(yes_path)
noes = os.listdir(no_path)

random.shuffle(yeses)
random.shuffle(noes)

samples_to_load = 100000
yeses = yeses[:samples_to_load]
noes = noes[:samples_to_load]

labels = {
    'yes' : 1,
    'no' : 0
}

train_x = []
train_y = []
train_test_div = len(yeses) #int(1.0*len(yeses))

for file in yeses[:train_test_div]:
    train_x.append(load_gray_img(yes_path+file))
    train_y.append(1)
    if len(train_x) % 1000 == 0:
        print(len(train_x))
print('train - yes done')
for file in noes[:train_test_div]:
    train_x.append(load_gray_img(no_path+file))
    train_y.append(0)
    if len(train_x) % 1000 == 0:
        print(len(train_x))
print('train - no done')
    
train_x = np.array(train_x, dtype=np.float32)
train_x /= 255.0
train_x = train_x.reshape(-1, train_x.shape[1], train_x.shape[2], 1)

print(train_x.shape, np.array(train_y).shape)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
train - yes done
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000

In [3]:
from keras.utils import to_categorical
train_y_cat = to_categorical(train_y)

train_y_cat.shape

Using TensorFlow backend.


NameError: name 'core' is not defined

In [ ]:
from sklearn.model_selection import train_test_split
print(train_x.shape, train_y_cat.shape)
train_x, valid_x, train_label, valid_label = train_test_split(train_x, train_y_cat, test_size=0.2, random_state=13)

train_x.shape, valid_x.shape

In [ ]:
import keras
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU

batch_size = 48
epochs = 30
num_classes = len(labels)
input_shape = train_x[0].shape

fashion_model = Sequential()

fashion_model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',padding='same',input_shape=input_shape))
fashion_model.add(Dropout(0.1))
fashion_model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',padding='same',input_shape=input_shape))
fashion_model.add(MaxPooling2D((2, 2),padding='same'))

fashion_model.add(Conv2D(64, (3, 3), activation='relu',padding='same'))
fashion_model.add(Dropout(0.2))
fashion_model.add(Conv2D(64, (3, 3), activation='relu',padding='same'))
fashion_model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))

fashion_model.add(Conv2D(128, (3, 3), activation='relu',padding='same'))
fashion_model.add(Dropout(0.3))
fashion_model.add(Conv2D(128, (3, 3), activation='relu',padding='same'))
fashion_model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))

fashion_model.add(Flatten())
fashion_model.add(Dense(128, activation='linear'))
fashion_model.add(LeakyReLU(alpha=0.1))           
fashion_model.add(Dropout(0.3))
fashion_model.add(Dense(num_classes, activation='softmax'))

fashion_model.compile(loss=keras.losses.categorical_crossentropy, 
                      optimizer='sgd',
                      metrics=['accuracy'])


fashion_model.summary()

fashion_train = fashion_model.fit(train_x, train_label, 
                                  batch_size=batch_size,
                                  epochs=epochs,
                                  verbose=1,
                                  shuffle=True,
                                  validation_data=(valid_x, valid_label))
fashion_model.save("filter_model.h5")
